# Data Cleaning Process

## Introduction

#### In this notebook, we will demonstrate the data cleaning process applied to a dataset.
#### The primary objectives are to handle missing values, correct data inconsistencies, and ensure data quality.

## Overview

####  We will use a  dataset containing vehicle information scraped from kai and karo website with columns such as Model,Price,Year,Transimission_Type,Usage Origin,CC.

## Objective
#### Our goal is to clean and prepare this data for further analysis.


In [1]:
# Import necessary libraries
import pandas as pd
from datetime import date

In [2]:
# Load the dataset
data = pd.read_csv('kai_karo_cars.csv')

# Display the first few rows of the dataframe
data.head(3)

,Model,Transimission_Type,Year,CC,Usage Origin,Price
0,Honda Vezel,Automatic,2017,1500 CC,Foreign Used,"KES 2,689,999"
1,Toyota Fielder,Automatic,2017,1500 CC,Kenyan Used,"KES 1,689,999"
2,Mercedes Benz C200,Automatic,2010,1800 CC,Kenyan Used,"KES 1,499,999"


## Initial Data Exploration

In [3]:
# Check for missing values
missing = data.isnull().sum()
missing

Model                 0
Transimission_Type    0
Year                  0
CC                    0
Usage Origin          0
Price                 0
dtype: int64

In [4]:
# Check for duplicate values
duplicates = data.duplicated().sum()
duplicates

44

In [5]:
# Visualize the duplicated rows
row_duplicates = data[data.duplicated(keep = False)]
row_duplicates

,Model,Transimission_Type,Year,CC,Usage Origin,Price
18,Toyota Landcruiser Prado TX,Automatic,2007,2700 CC,Kenyan Used,"KES 2,199,999"
21,Toyota Landcruiser Prado TX,Automatic,2007,2700 CC,Kenyan Used,"KES 2,199,999"
24,BMW 530i,Automatic,2017,2000 CC,Foreign Used,"KES 4,599,999"
54,BMW 118i,Automatic,2017,1500 CC,Foreign Used,"KES 2,349,999"
56,Subaru Forester XT,Automatic,2013,2000 CC,Kenyan Used,"KES 2,299,999"
...,...,...,...,...,...,...
1368,Audi A4,Automatic,2016,2000 CC,Foreign Used,"KES 3,150,000"
1376,Toyota Harrier,Automatic,2017,2000 CC,Foreign Used,"KES 3,850,000"
1389,Toyota Harrier,Automatic,2017,2000 CC,Foreign Used,"KES 3,850,000"
1390,Toyota Vanguard,Automatic,2012,2400 CC,Kenyan Used,"KES 1,899,999"


In [6]:
# Display values under price that are not in KES
other_currency = data[~data['Price'].str.contains('KES')]
other_currency

,Model,Transimission_Type,Year,CC,Usage Origin,Price
1436,Mercedes-Benz E350 D,Automatic,2017,3000 CC,Foreign Used,"£26,052"


In [7]:
# validate distinct transmission types
dist_trans = data['Transimission_Type'].unique()
dist_trans

array(['Automatic', 'Manual'], dtype=object)

In [8]:
# Check values without 'CC' in CC column
not_cc = data[~data['CC'].str.contains('CC')]
not_cc

,Model,Transimission_Type,Year,CC,Usage Origin,Price
1470,Isuzu Elf Light Truck,Automatic,2014,Foreign Used,Foreign Used,"KES 1,850,000"


In [9]:
# Validate Usage Origin Column
dist_origins = data['Usage Origin'].unique()
dist_origins

array(['Foreign Used', 'Kenyan Used'], dtype=object)

In [10]:
# Validate the datatypes
data.dtypes

Model                 object
Transimission_Type    object
Year                   int64
CC                    object
Usage Origin          object
Price                 object
dtype: object

In [11]:
# Summary statistics
data.describe()

,Year
count,1472.000000
mean,2014.000679
std,4.123106
min,1979.000000
25%,2013.000000
50%,2016.000000
75%,2016.000000
max,2024.000000


## Data Cleaning Steps

#### 1: Correcting data inconsistencies

#### (i) Price

In [12]:
other_currency = data[~data['Price'].str.contains('KES')]
other_currency

,Model,Transimission_Type,Year,CC,Usage Origin,Price
1436,Mercedes-Benz E350 D,Automatic,2017,3000 CC,Foreign Used,"£26,052"


In [13]:
# Converting column price to numeric values
data['Price'] = data['Price'].str.replace(r'[^0-9.]','',regex = True).astype('float')
# Display the first few rows of the dataframe
data.head(3)

,Model,Transimission_Type,Year,CC,Usage Origin,Price
0,Honda Vezel,Automatic,2017,1500 CC,Foreign Used,2689999.0
1,Toyota Fielder,Automatic,2017,1500 CC,Kenyan Used,1689999.0
2,Mercedes Benz C200,Automatic,2010,1800 CC,Kenyan Used,1499999.0


In [14]:
# Converting one price that is in euros to kes
data.at[1436, 'Price'] *= 167.21

In [15]:
# Checking if it has been updated
row = data.iloc[[1436]]
row

,Model,Transimission_Type,Year,CC,Usage Origin,Price
1436,Mercedes-Benz E350 D,Automatic,2017,3000 CC,Foreign Used,4356154.92


#### (ii) CC

In [16]:
# Converting CC column to numeric values
data['CC'] = data['CC'].replace(r'[^0-9]','',regex = True)
data['CC'] = pd.to_numeric(data['CC'] , errors = 'coerce')
# Display the first few rows of the dataframe
data.head()

,Model,Transimission_Type,Year,CC,Usage Origin,Price
0,Honda Vezel,Automatic,2017,1500.0,Foreign Used,2689999.0
1,Toyota Fielder,Automatic,2017,1500.0,Kenyan Used,1689999.0
2,Mercedes Benz C200,Automatic,2010,1800.0,Kenyan Used,1499999.0
3,Toyota Kluger,Automatic,2003,3500.0,Kenyan Used,879999.0
4,Mercedes Benz C250,Automatic,2017,2000.0,Foreign Used,4649999.0


In [17]:
# The value 'Foreign Used' under CC is converted to null by default

#### (iii) Transmission Type

In [18]:
data.rename(columns = {'Transimission_Type':'Transmission_Type'}, inplace = True)

#### 2:Removing duplicates

In [19]:
# Drop the duplicated rows
data = data.drop_duplicates()

# Confirming if duplicates are dropped
duplicates = data.duplicated().sum()
duplicates

0

In [20]:
# Checking length of data after removing duplicates
len(data)

1428

## Adding new columns

#### (i) Manufacturer and Car_Model

In [21]:
# Splitting column Model to Manufacturer and Model

data['Model'] = data['Model'].str.title()
unique_makes = ['Alfa Romea','Aston Martin','Land Rover','Mercedes Benz','Rolls Royce','Range Rover']

def model_split(Model):
    for unique_make in unique_makes:
        if Model.startswith(unique_make):
            Manufacturer = unique_make
            Car_Model = Model[len(Manufacturer):].strip()
            return Manufacturer, Car_Model

    Manufacturer_Model = Model.split(' ',1)
    # Handling cases without spaces
    if len(Manufacturer_Model) > 1:
        Manufacturer, Car_Model = Manufacturer_Model[0], Manufacturer_Model[1]
    else:
        Manufacturer = Model
        Car_Model = ' '
    return Manufacturer, Car_Model

data[['Manufacturer','Car_Model']] = data['Model'].apply(lambda x: pd.Series(model_split(x)))
# Display the first few rows of the dataframe
data.head(3)

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model
0,Honda Vezel,Automatic,2017,1500.0,Foreign Used,2689999.0,Honda,Vezel
1,Toyota Fielder,Automatic,2017,1500.0,Kenyan Used,1689999.0,Toyota,Fielder
2,Mercedes Benz C200,Automatic,2010,1800.0,Kenyan Used,1499999.0,Mercedes Benz,C200


#### (ii) Car_Age

In [22]:
date = date.today()
year = date.year

data['Car Age'] = data['Year'].apply(lambda x: year - x)
# Display the first few rows of the dataframe
data.head(3)

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age
0,Honda Vezel,Automatic,2017,1500.0,Foreign Used,2689999.0,Honda,Vezel,7
1,Toyota Fielder,Automatic,2017,1500.0,Kenyan Used,1689999.0,Toyota,Fielder,7
2,Mercedes Benz C200,Automatic,2010,1800.0,Kenyan Used,1499999.0,Mercedes Benz,C200,14


#### (iii) Price Range

In [23]:
def price_range(Price):
    if Price <= 500000:
        return '0-500k'
    elif Price <= 1000000:
        return '500k-1M'
    elif Price <= 2000000:
        return '1M-2M'
    elif Price <= 3000000:
        return '2M-3M'
    elif Price <= 5000000:
        return '3M-5M'
    elif Price <=10000000:
        return '5M-10M'
    else:
        return 'Above 10'

data['Price Range'] = data['Price'].apply(lambda x: price_range(x))

# Display the first few rows of the dataframe
data.head(3)

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
0,Honda Vezel,Automatic,2017,1500.0,Foreign Used,2689999.0,Honda,Vezel,7,2M-3M
1,Toyota Fielder,Automatic,2017,1500.0,Kenyan Used,1689999.0,Toyota,Fielder,7,1M-2M
2,Mercedes Benz C200,Automatic,2010,1800.0,Kenyan Used,1499999.0,Mercedes Benz,C200,14,1M-2M


## Final Data Exploration

In [24]:
# Distinct manufacturers
unique_manufacturer = data['Manufacturer'].unique()
for make in unique_manufacturer:
    print(make)
print("\n Number of manufacturers:",len(unique_manufacturer))

Honda
Toyota
Mercedes Benz
Audi
Bmw
Landrover
Nissan
Mitsubishi
Suzuki
Mazda
Volkswagen
Volvo
Lexus
Daihatsu
Subaru
Ford
Smart
Jeep
Peugeot
Range Rover
Porsche
Jaguar
Kia
Hyundai
Renault
Mini
Jincheng
Yamaha
Isuzu
Land Rover
Kawasaki
Citreon
Dodge
Citroën
Faw
Chevrolet
Alfa
Landcruiser
Hino
Passo
Tata
Mercedes-Benz

 Number of manufacturers: 42


## Final Cleaning

#### (i) Manufacturer

In [25]:
# Removing land cruiser under manufacturer and replacing with toyota
check = data[data['Manufacturer'] == 'Landcruiser']
check

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
1130,"Landcruiser Prado J120 ""1Kd""/Sunroof",Automatic,2007,3000.0,Kenyan Used,1999999.0,Landcruiser,"Prado J120 ""1Kd""/Sunroof",17,1M-2M


In [26]:
# Prepending the string 'Toyota'
data.at[1130, 'Model'] = 'Toyota ' + data.at[1130, 'Model']

In [27]:
# Checking if prepended
check = data[data['Manufacturer'] == 'Landcruiser']
check

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
1130,"Toyota Landcruiser Prado J120 ""1Kd""/Sunroof",Automatic,2007,3000.0,Kenyan Used,1999999.0,Landcruiser,"Prado J120 ""1Kd""/Sunroof",17,1M-2M


In [28]:
# Removing the dash in "Mercedez benz" and replacing with a dash
check = data[data['Manufacturer'] == 'Mercedes-Benz']
check

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
1436,Mercedes-Benz E350 D,Automatic,2017,3000.0,Foreign Used,4356154.92,Mercedes-Benz,E350 D,7,3M-5M


In [29]:
# Check if removed
data.loc[1436, 'Model'] = data.loc[1436, 'Model'].replace('-', ' ')
check = data[data['Manufacturer'] == 'Mercedes-Benz']
check

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
1436,Mercedes Benz E350 D,Automatic,2017,3000.0,Foreign Used,4356154.92,Mercedes-Benz,E350 D,7,3M-5M


In [30]:
# Splitting the name "landrover" into "land rover"
data['Model'] = data['Model'].str.replace('Landrover', 'Land Rover', regex=False)

# Check if removed
check = data[data['Manufacturer'] == 'Landrover']
check

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
7,Land Rover Discovery V,Automatic,2017,3000.0,Foreign Used,9499999.0,Landrover,Discovery V,7,5M-10M
47,Land Rover Defender 110,Automatic,2024,3000.0,Foreign Used,22999999.0,Landrover,Defender 110,0,Above 10
58,Land Rover Discovery Iv,Automatic,2013,3000.0,Kenyan Used,4549999.0,Landrover,Discovery Iv,11,3M-5M
218,Land Rover Discovery Iv,Automatic,2013,3000.0,Kenyan Used,4499999.0,Landrover,Discovery Iv,11,3M-5M
271,Land Rover Discovery V,Automatic,2017,3000.0,Kenyan Used,9999999.0,Landrover,Discovery V,7,5M-10M
368,Land Rover Defender 110,Automatic,2021,3000.0,Kenyan Used,18999999.0,Landrover,Defender 110,3,Above 10
440,Land Rover Defender 110,Automatic,1990,2500.0,Kenyan Used,1699999.0,Landrover,Defender 110,34,1M-2M
695,Land Rover Discovery Iv,Automatic,2012,3000.0,Kenyan Used,3999999.0,Landrover,Discovery Iv,12,3M-5M
785,Land Rover Defender 110,Manual,2011,2500.0,Kenyan Used,4099999.0,Landrover,Defender 110,13,3M-5M
847,Land Rover Discovery Iv / 3-Zone Sunroofs,Automatic,2012,3000.0,Kenyan Used,3999999.0,Landrover,Discovery Iv / 3-Zone Sunroofs,12,3M-5M


In [31]:
# Rerun the code to correct the modified values
# Splitting column Model to Manufacturer and Model

data['Model'] = data['Model'].str.title()
unique_makes = ['Alfa Romea','Aston Martin','Land Rover','Mercedes Benz','Rolls Royce','Range Rover']

def model_split(Model):
    for unique_make in unique_makes:
        if Model.startswith(unique_make):
            Manufacturer = unique_make
            Car_Model = Model[len(Manufacturer):].strip()
            return Manufacturer, Car_Model

    Manufacturer_Model = Model.split(' ',1)
    # Handling cases without spaces
    if len(Manufacturer_Model) > 1:
        Manufacturer, Car_Model = Manufacturer_Model[0], Manufacturer_Model[1]
    else:
        Manufacturer = Model
        Car_Model = ' '
    return Manufacturer, Car_Model

data[['Manufacturer','Car_Model']] = data['Model'].apply(lambda x: pd.Series(model_split(x)))
data.head(3)

,Model,Transmission_Type,Year,CC,Usage Origin,Price,Manufacturer,Car_Model,Car Age,Price Range
0,Honda Vezel,Automatic,2017,1500.0,Foreign Used,2689999.0,Honda,Vezel,7,2M-3M
1,Toyota Fielder,Automatic,2017,1500.0,Kenyan Used,1689999.0,Toyota,Fielder,7,1M-2M
2,Mercedes Benz C200,Automatic,2010,1800.0,Kenyan Used,1499999.0,Mercedes Benz,C200,14,1M-2M


In [32]:
# Distinct manufacturers
unique_manufacturer = data['Manufacturer'].unique()
for make in unique_manufacturer:
    print(make)
print("\n Number of manufacturers:",len(unique_manufacturer))

Honda
Toyota
Mercedes Benz
Audi
Bmw
Land Rover
Nissan
Mitsubishi
Suzuki
Mazda
Volkswagen
Volvo
Lexus
Daihatsu
Subaru
Ford
Smart
Jeep
Peugeot
Range Rover
Porsche
Jaguar
Kia
Hyundai
Renault
Mini
Jincheng
Yamaha
Isuzu
Kawasaki
Citreon
Dodge
Citroën
Faw
Chevrolet
Alfa
Hino
Passo
Tata

 Number of manufacturers: 39


##### NB: The number of manufactures has reduced from 42 to 39

In [33]:
# Final Summary statistics
data.describe()

,Year,CC,Price,Car Age
count,1428.000000,1427.000000,1.428000e+03,1428.000000
mean,2013.943978,2063.281710,2.757793e+06,10.056022
std,4.160878,727.408256,2.156263e+06,4.160878
min,1979.000000,350.000000,2.199990e+05,0.000000
25%,2013.000000,1500.000000,1.329999e+06,8.000000
50%,2016.000000,2000.000000,2.199999e+06,8.000000
75%,2016.000000,2500.000000,3.549999e+06,11.000000
max,2024.000000,5700.000000,2.300000e+07,45.000000


In [34]:
# Confirming the data types
data.dtypes

Model                 object
Transmission_Type     object
Year                   int64
CC                   float64
Usage Origin          object
Price                float64
Manufacturer          object
Car_Model             object
Car Age                int64
Price Range           object
dtype: object

In [35]:
# Editing Column names
data.rename(columns = {'Model':'Vehicle','Transmission_Type':'Transmission Type','Car_Model':'Model'}, inplace = True)
data.head(3)

,Vehicle,Transmission Type,Year,CC,Usage Origin,Price,Manufacturer,Model,Car Age,Price Range
0,Honda Vezel,Automatic,2017,1500.0,Foreign Used,2689999.0,Honda,Vezel,7,2M-3M
1,Toyota Fielder,Automatic,2017,1500.0,Kenyan Used,1689999.0,Toyota,Fielder,7,1M-2M
2,Mercedes Benz C200,Automatic,2010,1800.0,Kenyan Used,1499999.0,Mercedes Benz,C200,14,1M-2M


In [36]:
# Rearranging columns
data = data[['Vehicle','Manufacturer','Model','Transmission Type','CC','Year','Car Age','Usage Origin','Price','Price Range']]
data.head(3)

,Vehicle,Manufacturer,Model,Transmission Type,CC,Year,Car Age,Usage Origin,Price,Price Range
0,Honda Vezel,Honda,Vezel,Automatic,1500.0,2017,7,Foreign Used,2689999.0,2M-3M
1,Toyota Fielder,Toyota,Fielder,Automatic,1500.0,2017,7,Kenyan Used,1689999.0,1M-2M
2,Mercedes Benz C200,Mercedes Benz,C200,Automatic,1800.0,2010,14,Kenyan Used,1499999.0,1M-2M


In [37]:
# Striping leading and trailing white spaces in columns
data['Vehicle'] = data['Vehicle'].str.strip()
data['Transmission Type'] = data['Transmission Type'].str.strip()
data['Usage Origin'] = data['Usage Origin'].str.strip()
data.head(3)

,Vehicle,Manufacturer,Model,Transmission Type,CC,Year,Car Age,Usage Origin,Price,Price Range
0,Honda Vezel,Honda,Vezel,Automatic,1500.0,2017,7,Foreign Used,2689999.0,2M-3M
1,Toyota Fielder,Toyota,Fielder,Automatic,1500.0,2017,7,Kenyan Used,1689999.0,1M-2M
2,Mercedes Benz C200,Mercedes Benz,C200,Automatic,1800.0,2010,14,Kenyan Used,1499999.0,1M-2M


## Saving as a csv file

In [38]:
data.to_csv('Kai & Karo_clean_data.csv', index = False)